### Login the reporting server to download the files. 

### From here, we go by mannually to downloads these reports, becasue this servce doesn't allow to look inspects.  (▰˘︹˘ ▰)

### Read all the 4 reports by using partial names

In [1]:
import glob

import pandas as pd
import numpy as np
import csv
from datetime import datetime

#Now reading partial names as list and loading them as one one. 

partial_names = ["Cybex General Open Orders (Routing Tool)", 
                 "Shipping Totals by Order Line",
                 "From Dock to Stock Report", 
                 "Inventory Cycle Counts"]
files = []
#collect it to the list... 

for name in partial_names:
    files.extend(glob.glob(r"C:\Users\liuchen\Downloads\*{}*".format(name)))
    
#all good part, don't do anything!!

In [2]:
#read all the files as dataframe!! 

for file in files:
    if "Cybex General Open Orders (Routing Tool)" in file: 
        tool = pd.read_excel(file)
    elif "Shipping Totals by Order Line" in file:
        ship = pd.read_excel(file)
    elif "From Dock to Stock Report" in file:
        stock = pd.read_excel(file)
    elif "Inventory Cycle Counts" in file:
        inventory = pd.read_excel(file)
        
## all good part, don't do anything!! can be read as usual dataframe.
## Do NOT open the file when the python running!

C:\Users\liuchen\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\liuchen\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\liuchen\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\liuchen\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
ship['Order Type'].unique()

array(['B2B-RETAIL', 'B2B-ORDER', 'B2B-INTERNATIONAL',
       'B2B-INTERNATIONAL RETAIL', 'eCOM', 'B2B-DROPSHIP', 'Not Defined'],
      dtype=object)

### (╭☞•̀o•́)╭☞This is column C/D B2B Retail 

In [4]:
from datetime import datetime, timedelta

yesterday = datetime.now() - timedelta(days=1)

In [5]:
yesterday 

datetime.datetime(2023, 4, 12, 13, 30, 48, 776738)

In [6]:
ship['Order Type'].unique()

array(['B2B-RETAIL', 'B2B-ORDER', 'B2B-INTERNATIONAL',
       'B2B-INTERNATIONAL RETAIL', 'eCOM', 'B2B-DROPSHIP', 'Not Defined'],
      dtype=object)

In [7]:
column_c_d = ship[ship['Order Type'].isin(['B2B-RETAIL', 'B2B-ORDER', 'B2B-INTERNATIONAL', 'B2B-INTERNATIONAL RETAIL'])]

In [8]:
column_c_d = column_c_d[column_c_d['Loaded Date'].dt.date == yesterday.date()]

#pick only the date without timestamp! very important! with time will need more conditions. 

In [9]:
print (f"\n ** the B2B ON time Orders: {column_c_d['Order'].nunique()} **\n")

# Done! 


 ** the B2B ON time Orders: 57 **



### (╭☞•̀o•́)╭☞This is column G/H Ecom outbound on time Orders

In [10]:
ecom = ship[ship['Order Type'].isin(['eCOM'])]

In [11]:
from datetime import datetime, timedelta

start_date = datetime.now() - timedelta(days=4)
middle_date = datetime.now() - timedelta(days=3)
end_date = datetime.now() - timedelta(days=1)
target_time = datetime.strptime('14:00:00', '%H:%M:%S').time()

In [12]:
start_date

datetime.datetime(2023, 4, 9, 13, 30, 48, 884784)

In [13]:
middle_date

datetime.datetime(2023, 4, 10, 13, 30, 48, 884784)

In [14]:
end_date

datetime.datetime(2023, 4, 12, 13, 30, 48, 884784)

In [15]:
if datetime.now().weekday() == 0:
    # 如果今天是星期一,星期四<->星期五
    ecom_order= ecom[(ecom['WMS Add Date'] >= str(start_date.date()) + ' ' + str(target_time)) 
                             & (ecom['WMS Add Date'] < str(middle_date.date() + timedelta(days=1)) + ' ' + str(target_time))]
    
elif datetime.now().weekday() == 1:
    # 如果今天是周二，星期5<->星期一，+周末
    ecom_order= ecom[(ecom['WMS Add Date'] >= str(start_date.date()) + ' ' + str(target_time)) 
                             & (ecom['WMS Add Date'] < str(end_date.date()) + ' ' + str(target_time))]
else:
    # 如果今天是剩下周日，就应该是2天
    ecom_order= ecom[(ecom['WMS Add Date'] >= str(end_date.date() - timedelta(days=1)) + ' ' + str(target_time)) 
                             & (ecom['WMS Add Date'] <= str(end_date.date()) + ' ' + str(target_time))]

In [16]:
print (f"\n ** the eCOM outbound ON time Orders: {ecom_order['Order'].nunique()} **\n")

# Done! 


 ** the eCOM outbound ON time Orders: 25 **



### (╭☞•̀o•́)╭☞This is column K/L Dropship on time Orders

In [17]:
dropship = ship[ship['Order Type'].isin(['B2B-DROPSHIP'])]

In [18]:
from datetime import datetime, timedelta

dropship_start = datetime.now() - timedelta(days=5)
dropship_middle = datetime.now() - timedelta(days=4)
dropship_end = datetime.now() - timedelta(days=2)
target_time = datetime.strptime('14:00:00', '%H:%M:%S').time()

In [19]:
dropship_start

datetime.datetime(2023, 4, 8, 13, 30, 48, 995139)

In [20]:
dropship_middle

datetime.datetime(2023, 4, 9, 13, 30, 48, 995139)

In [21]:
dropship_end

datetime.datetime(2023, 4, 11, 13, 30, 48, 995139)

In [22]:
#this is 4 parts loop for different date requirement.
#if it is Mon, Tus, Wes and the rest of weeks. I do think there is better way but don't know yet

if datetime.now().weekday() == 0:
    # 如果今天是星期一,星期三<->星期四
    dropship_order = dropship[(dropship['WMS Add Date'] >= str(dropship_start.date()) + ' ' + str(target_time)) 
                             & (dropship['WMS Add Date'] < str(dropship_middle.date() + timedelta(days=1)) + ' ' + str(target_time))]

elif datetime.now().weekday() == 1:
    # 如果今天是星期二，星期四<->星期五
    dropship_order = dropship[(dropship['WMS Add Date'] >= str(dropship_start.date()) + ' ' + str(target_time)) 
                             & (dropship['WMS Add Date'] < str(dropship_midddle.date()) + ' ' + str(target_time))]
    
elif datetime.now().weekday() == 2:
    # 如果今天是星期三，星期五<->星期1 + 周末
    dropship_order = dropship[(dropship['WMS Add Date'] >= str(dropship_start.date()) + ' ' + str(target_time)) 
                             & (dropship['WMS Add Date'] < str(dropship_end.date()) + ' ' + str(target_time))]

else:
    # 如果今天是剩下周日，就应该是2天
    dropship_order = dropship[(dropship['WMS Add Date'] >= str(datetime.now() - timedelta(days=3)) + ' ' + str(target_time)) 
                             & (dropship['WMS Add Date'] < str(datetime.now() - timedelta(days=2)) + ' ' + str(target_time))]

In [23]:
print (f"\n ** the B2C dropship ON time Orders: {dropship_order['Order'].nunique()} **\n")


 ** the B2C dropship ON time Orders: 21 **



### if yesterday apperead in the list, then it will give the answer and also, you can pick the date to check

In [24]:
# Convert the 'Count Date' column to a Pandas DatetimeIndex
inventory['Count Date'] = pd.to_datetime(inventory['Count Date'])

# Use strftime() to convert the dates to the desired format
inventory['Date'] = inventory['Count Date'].dt.strftime('%Y/%m/%d')

# Print the unique count dates in the new format
unique_dates = inventory['Date'].unique()

#unique_dates = pd.DataFrame(unique_dates)

#unique_dates.columns = ['date']

unique_dates

# must create new column just date, becasue if it with time. It can't be called. 

array(['2023/03/13', '2023/03/16', '2023/03/17', '2023/03/30',
       '2023/03/31', '2023/04/04', '2023/04/06', '2023/04/07',
       '2023/04/10', '2023/04/12'], dtype=object)

In [25]:
# Get the current date and subtract one day
date_to_pick = datetime.now() - timedelta(days=1)

# Convert date_to_pick to string in the same format as unique_dates
date_to_pick_str = date_to_pick.strftime('%Y/%m/%d')

# Initialize flag to check if date_to_pick is found in unique_dates
date_found = False

# Check if date_to_pick is in unique_dates
for unique_date in unique_dates:
    if date_to_pick_str in unique_date:
        filtered_inventory = inventory[(inventory['Count Type'] != 'Audit Count') & (inventory['Date'] == date_to_pick_str)]
        num_unique_locations = filtered_inventory["Location"].nunique()
        sum_counted_qty = int(filtered_inventory["Counted Qty"].sum())
        sum_expected_qty = int(filtered_inventory["Expected Qty"].sum())
        date_found = True
        break

# If date_to_pick is found in unique_dates, print the results, otherwise print "There is no data for today"
if date_found:
    print(f"\n*** The Unique Locations # for {date_to_pick_str} is {num_unique_locations} ***")
    print(f"\n*** The Sum of Actual Inventory # for {date_to_pick_str} is {sum_counted_qty} ***")
    print(f"\n*** The Sum of Expected Inventory # for {date_to_pick_str} is {sum_expected_qty} ***")
else:
    print("There is no data for today")



*** The Unique Locations # for 2023/04/12 is 237 ***

*** The Sum of Actual Inventory # for 2023/04/12 is 2098 ***

*** The Sum of Expected Inventory # for 2023/04/12 is 2098 ***


In [26]:
# order['WMS Add Date'].dtype = object
ship['Loaded Date'] = pd.to_datetime(ship['Loaded Date'])

# Use strftime() to convert the dates to the desired format
ship['Loaded Date'] = ship['Loaded Date'].dt.strftime('%Y/%m/%d')

# Print the unique count dates in the new format
unique_dates = ship['Loaded Date'].unique()
unique_dates = sorted(unique_dates, reverse=False)
print(unique_dates)

['2023/02/23', '2023/02/24', '2023/02/27', '2023/02/28', '2023/03/01', '2023/03/02', '2023/03/03', '2023/03/06', '2023/03/07', '2023/03/08', '2023/03/09', '2023/03/10', '2023/03/13', '2023/03/14', '2023/03/15', '2023/03/16', '2023/03/17', '2023/03/20', '2023/03/21', '2023/03/22', '2023/03/23', '2023/03/24', '2023/03/27', '2023/03/28', '2023/03/29', '2023/03/30', '2023/03/31', '2023/04/03', '2023/04/04', '2023/04/05', '2023/04/06', '2023/04/07', '2023/04/10', '2023/04/11', '2023/04/12']


In [27]:
date_to_pick = input("Enter a date (YYYY-MM-DD) to check the inventory")


sum_of_outbound = ship[(ship['Order Type'] != 'Not Defined') & (ship['Loaded Date'] == date_to_pick)]
outbound_order_sum = sum_of_outbound['Order'].nunique()
    
print(f"\n\n***  The Total Outbound Orders Completed # for {date_to_pick} is ** {outbound_order_sum} **")

#Column O

Enter a date (YYYY-MM-DD) to check the inventory2023/04/12


***  The Total Outbound Orders Completed # for 2023/04/12 is ** 106 **


In [33]:
#this is the loop for checking any kind of date for the inventory unique# without desotry the "inventory" dataframe
#这个loop不会只显示一个时间段，这个loop可以自由选择时间来得到数据

while True:
    date_to_pick = input("Enter a date (YYYY-MM-DD) to check the inventory or 'd' to quit: ")
    if date_to_pick == 'd':
        break
        
    filtered_inventory = inventory[(inventory['Count Type'] != 'Audit Count') & (inventory['Date'] == date_to_pick)]
    num_unique_locations = filtered_inventory["Location"].nunique()
    
    print(f"***  The Unique Locations # for {date_to_pick} is ** {num_unique_locations}  ***")
    
    sum_counted_qty = int(filtered_inventory["Counted Qty"].sum())
    
    print(f"***  The Sum of Actual Inventory # for {date_to_pick} is ** {sum_counted_qty}  ***")
    
    sum_expected_qty = int(filtered_inventory["Expected Qty"].sum())
    
    print(f"***  The Sum of Expected Inventory # for {date_to_pick} is ** {sum_expected_qty}  ***")
    
#Column AG, AI, AJ, all this part is good. if there is no data, which is 0, Do not change anything!!

Enter a date (YYYY-MM-DD) to check the inventory or 'd' to quit: 2023/04/12
***  The Unique Locations # for 2023/04/12 is ** 237  ***
***  The Sum of Actual Inventory # for 2023/04/12 is ** 2098  ***
***  The Sum of Expected Inventory # for 2023/04/12 is ** 2098  ***
Enter a date (YYYY-MM-DD) to check the inventory or 'd' to quit: d


In [31]:
print ("GOOD work!! BOY!")

GOOD work!! BOY!
